# 1.Crop dataset

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Define function to load images, detect faces, and crop them with margin
def detect_and_crop_faces(base_path, output_path, margin=0.2):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for person_name in os.listdir(base_path):
        person_path = os.path.join(base_path, person_name)
        if os.path.isdir(person_path):
            # Create output directory for each person
            person_output_path = os.path.join(output_path, person_name)
            os.makedirs(person_output_path, exist_ok=True)

            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                try:
                    # Read image
                    image = cv2.imread(image_path)
                    if image is None:
                        print(f"Skipping {image_path}, not a valid image.")
                        continue
                    
                    # Convert the image to RGB as MediaPipe expects RGB input
                    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
                    # Detect faces
                    results = face_detection.process(image_rgb)

                    if results.detections:
                        for i, detection in enumerate(results.detections):
                            # Get bounding box coordinates
                            bboxC = detection.location_data.relative_bounding_box
                            h, w, _ = image.shape
                            
                            # Calculate margins
                            x_min = int((bboxC.xmin - margin) * w)
                            y_min = int((bboxC.ymin - margin) * h)
                            x_max = int((bboxC.xmin + bboxC.width + margin) * w)
                            y_max = int((bboxC.ymin + bboxC.height + margin) * h)

                            # Ensure the coordinates are within image bounds
                            x_min = max(0, x_min)
                            y_min = max(0, y_min)
                            x_max = min(w, x_max)
                            y_max = min(h, y_max)

                            # Crop the face with margin
                            cropped_face = image[y_min:y_max, x_min:x_max]
                            face_gray = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)

                            # Save the cropped face
                            output_image_path = os.path.join(
                                person_output_path, f"{os.path.splitext(image_name)[0]}_face_{i}.jpg"
                            )
                            cv2.imwrite(output_image_path, face_gray)
                            print(f"Saved cropped face to {output_image_path}")
                    else:
                        print(f"No face detected in {image_path}")

                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

# Example usage
base_path = "dataset"
output_path = "new_grayscale"
detect_and_crop_faces(base_path, output_path, margin=0.2)


In [ ]:
# Crop one image

import os
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

output_dir = "testing_data"
os.makedirs(output_dir, exist_ok=True)

image_path = "20241122_173529.jpg" 
image = cv2.imread(image_path)

if image is None:
    print(f"Error: Image file '{image_path}' not found.")
else:
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform face detection
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(rgb_image)

        if results.detections:
            for i, detection in enumerate(results.detections):
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = image.shape
                margin = 0.2
                x_min = int((bboxC.xmin - margin) * w)
                y_min = int((bboxC.ymin - margin) * h)
                x_max = int((bboxC.xmin + bboxC.width + margin) * w)
                y_max = int((bboxC.ymin + bboxC.height + margin) * h)

                x_min = max(0, x_min)
                y_min = max(0, y_min)
                x_max = min(w, x_max)
                y_max = min(h, y_max)

                cropped_face = image[y_min:y_max, x_min:x_max]
                face_gray = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)
                
                output_path = os.path.join(output_dir, f"testing_image_{i + 4}.jpg")
                cv2.imwrite(output_path, face_gray)
                print(f"Saved cropped face {i + 1} to '{output_path}'")
        else:
            print("No faces detected.")


# 2.Training Triplet loss model

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.spatial.distance import cdist

# Parameters
IMAGE_SIZE = (112, 112)
DATASET_PATH = "new_grayscale/"

# Function to load and label images based on folder structure
def load_images_from_directory(base_path):
    data = []
    labels = []
    label_map = {}
    label_counter = 0

    # Loop through each employee folder
    for person_name in os.listdir(base_path):
        person_path = os.path.join(base_path, person_name)
        if os.path.isdir(person_path):
            if person_name not in label_map:
                label_map[person_name] = label_counter
                label_counter += 1

            # Load all images in the person's folder
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                try:
                    image = cv2.imread(image_path)
                    if image is not None:
                        image = cv2.resize(image, IMAGE_SIZE)
                        data.append(image)
                        labels.append(label_map[person_name])
                except Exception as e:
                    print(f"Could not read image {image_path}: {e}")

    return np.array(data), np.array(labels), label_map

# Load dataset from the folder structure
images, labels, label_map = load_images_from_directory(DATASET_PATH)

# Normalize images (scaling pixel values to [0, 1])
images = images / 255.0

print("Loaded images:", len(images))
print("Unique labels:", len(set(labels)))

# Split dataset into training and validation
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Model definition (Siamese-style network for triplet loss)
def create_embedding_model(input_shape=IMAGE_SIZE + (3,)):
    input = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    embedding = layers.Dense(64)(x)
    model = Model(input, embedding)
    return model

@tf.keras.utils.register_keras_serializable()
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    positive_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    negative_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(positive_dist - negative_dist + alpha, 0.0)
    return tf.reduce_mean(loss)

# Register custom loss function
from tensorflow.keras.utils import get_custom_objects
get_custom_objects()['triplet_loss'] = triplet_loss

# Build the model
embedding_model = create_embedding_model()

# Triplet model
anchor_input = layers.Input(shape=IMAGE_SIZE + (3,))
positive_input = layers.Input(shape=IMAGE_SIZE + (3,))
negative_input = layers.Input(shape=IMAGE_SIZE + (3,))

# Get embeddings
anchor_embedding = embedding_model(anchor_input)
positive_embedding = embedding_model(positive_input)
negative_embedding = embedding_model(negative_input)

# Concatenate the embeddings
merged_output = layers.concatenate([anchor_embedding, positive_embedding, negative_embedding], axis=-1)

# Define the model for triplet loss
triplet_model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)

# Compile the model with the triplet loss function
triplet_model.compile(optimizer=Adam(learning_rate=0.0001), loss=triplet_loss)

# Prepare triplets for training (anchor, positive, negative)
def generate_triplets(X, y, num_triplets=1000):
    anchors, positives, negatives = [], [], []
    for _ in range(num_triplets):
        # Pick a random class (person)
        label = np.random.choice(np.unique(y))
        # Get indices of the same class
        same_class_indices = np.where(y == label)[0]
        # Pick an anchor and positive image from the same class
        anchor_idx, positive_idx = np.random.choice(same_class_indices, 2, replace=False)
        # Pick a negative image from a different class
        different_class_indices = np.where(y != label)[0]
        negative_idx = np.random.choice(different_class_indices)

        # Append to triplet arrays
        anchors.append(X[anchor_idx])
        positives.append(X[positive_idx])
        negatives.append(X[negative_idx])

    return np.array(anchors), np.array(positives), np.array(negatives)

# Generate triplets for training
anchors, positives, negatives = generate_triplets(X_train, y_train, num_triplets=5000)

# Train the model on triplets
triplet_model.fit(
    [anchors, positives, negatives],
    np.zeros(len(anchors)),  # Labels are not used for triplet loss
    epochs=10,
    batch_size=35
)

# Save the trained model
triplet_model.save('model_grayscale/triplet_model.keras')
print("Triplet model saved to 'model_grayscale/triplet_model.keras'")

embedding_model.save('model_grayscale/embedding_model.keras')
print("Embedding model saved to 'model_grayscale/embedding_model.keras'")

# Validation Code
# Generate embeddings for the training and validation sets
train_embeddings = embedding_model.predict(X_train)
val_embeddings = embedding_model.predict(X_val)

# Map training labels to their embeddings for easier lookup
train_embeddings_map = {label: train_embeddings[np.where(y_train == label)] for label in np.unique(y_train)}

# Predict validation labels using Nearest Neighbor classification
y_val_pred = []
for val_embedding in val_embeddings:
    # Calculate distances from the validation embedding to all training embeddings
    distances = {label: np.min(cdist([val_embedding], train_embeddings_map[label])) for label in train_embeddings_map}
    # Predict the label with the smallest distance
    predicted_label = min(distances, key=distances.get)
    y_val_pred.append(predicted_label)

# Compute validation accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.2f}")

# Classification report (precision, recall, F1-score)
print("Classification Report:")
print(classification_report(y_val, y_val_pred))

# Generate triplets for validation
val_anchors, val_positives, val_negatives = generate_triplets(X_val, y_val, num_triplets=500)

# Evaluate triplet loss on validation triplets
val_loss = triplet_model.evaluate([val_anchors, val_positives, val_negatives], np.zeros(len(val_anchors)), verbose=0)
print(f"Validation Loss (Triplet): {val_loss:.2f}")


2024-11-26 12:51:43.397943: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-26 12:51:43.400400: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 12:51:43.432517: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 12:51:43.471764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732605703.510596   46790 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732605703.52

Loaded images: 147
Unique labels: 10


2024-11-26 12:51:49.415534: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/10


/home/ritika/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_8', 'keras_tensor_9', 'keras_tensor_10']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


143/143 ━━━━━━━━━━━━━━━━━━━━ 46s 313ms/step - loss: 0.3978
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 311ms/step - loss: 0.1918
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 46s 325ms/step - loss: 0.1968
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 44s 306ms/step - loss: 0.2250
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 48s 337ms/step - loss: 0.2104
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 316ms/step - loss: 0.2017
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 312ms/step - loss: 0.1933
Epoch 8/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 45s 312ms/step - loss: 0.2007
Epoch 9/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 44s 308ms/step - loss: 0.2083
Epoch 10/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 44s 306ms/step - loss: 0.2047
Triplet model saved to 'model_grayscale/triplet_model.keras'
Embedding model saved to 'model_grayscale/embedding_model.keras'
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Validation Accuracy: 0.23
Classification Report:
              precision    recall  f1-score   support

    

/home/ritika/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ritika/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ritika/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ritika/anaconda

Validation Loss (Triplet): 0.20


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

# Parameters
IMAGE_SIZE = (112, 112)
DATASET_PATH = "new_grayscale/"

# Function to load and label images based on folder structure
def load_images_from_directory(base_path):
    data = []
    labels = []
    label_map = {}
    label_counter = 0

    # Loop through each employee folder
    for person_name in os.listdir(base_path):
        person_path = os.path.join(base_path, person_name)
        if os.path.isdir(person_path):
            if person_name not in label_map:
                label_map[person_name] = label_counter
                label_counter += 1

            # Load all images in the person's folder
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                try:
                    image = cv2.imread(image_path)
                    if image is not None:
                        image = cv2.resize(image, IMAGE_SIZE)
                        data.append(image)
                        labels.append(label_map[person_name])
                except Exception as e:
                    print(f"Could not read image {image_path}: {e}")

    return np.array(data), np.array(labels), label_map

# Load dataset from the folder structure
images, labels, label_map = load_images_from_directory(DATASET_PATH)

# Normalize images (scaling pixel values to [0, 1])
images = images / 255.0

print("Loaded images:", len(images))
print("Unique labels:", len(set(labels)))

# Split dataset into training and validation
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Model definition (Siamese-style network for triplet loss)
def create_embedding_model(input_shape=IMAGE_SIZE + (3,)):
    input = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    embedding = layers.Dense(64)(x)
    model = Model(input, embedding)
    return model

@tf.keras.utils.register_keras_serializable()
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    positive_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    negative_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(positive_dist - negative_dist + alpha, 0.0)
    return tf.reduce_mean(loss)

# Register custom loss function
from tensorflow.keras.utils import get_custom_objects
get_custom_objects()['triplet_loss'] = triplet_loss

# Build the model
embedding_model = create_embedding_model()

# Triplet model
anchor_input = layers.Input(shape=IMAGE_SIZE + (3,))
positive_input = layers.Input(shape=IMAGE_SIZE + (3,))
negative_input = layers.Input(shape=IMAGE_SIZE + (3,))

# Get embeddings
anchor_embedding = embedding_model(anchor_input)
positive_embedding = embedding_model(positive_input)
negative_embedding = embedding_model(negative_input)

# Concatenate the embeddings
merged_output = layers.concatenate([anchor_embedding, positive_embedding, negative_embedding], axis=-1)

# Define the model for triplet loss
triplet_model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=merged_output)

# Compile the model with the triplet loss function
triplet_model.compile(optimizer=Adam(learning_rate=0.0001), loss=triplet_loss)

# Prepare triplets for training (anchor, positive, negative)
def generate_triplets(X, y, num_triplets=1000):
    anchors, positives, negatives = [], [], []
    for _ in range(num_triplets):
        # Pick a random class (person)
        label = np.random.choice(np.unique(y))
        # Get indices of the same class
        same_class_indices = np.where(y == label)[0]
        # Pick an anchor and positive image from the same class
        anchor_idx, positive_idx = np.random.choice(same_class_indices, 2, replace=False)
        # Pick a negative image from a different class
        different_class_indices = np.where(y != label)[0]
        negative_idx = np.random.choice(different_class_indices)

        # Append to triplet arrays
        anchors.append(X[anchor_idx])
        positives.append(X[positive_idx])
        negatives.append(X[negative_idx])

    return np.array(anchors), np.array(positives), np.array(negatives)


# Generate triplets for training
anchors, positives, negatives = generate_triplets(X_train, y_train, num_triplets=5000)

# Train the model on triplets
triplet_model.fit(
    [anchors, positives, negatives],
    np.zeros(len(anchors)),  # Labels are not used for triplet loss
    epochs=10,
    batch_size=35
)

# Save the trained model
triplet_model.save('model_grayscale/triplet_model.keras')
print("Triplet model saved to 'model_grayscale/triplet_model.keras'")

embedding_model.save('model_grayscale/embedding_model.keras')
print("Embedding model saved to 'model_grayscale/embedding_model.keras'")

# Save embeddings for each employee during training
def generate_embeddings_for_all_employees(X, y, label_map, embedding_model):
    employee_embeddings = {}

    # Iterate over each unique employee (identified by labels)
    for label in np.unique(y):
        # Get all images for this employee
        employee_images = X[np.where(y == label)[0]]
        employee_name = list(label_map.keys())[list(label_map.values()).index(label)]
        
        embeddings = []
        
        # Extract embeddings for each image of the employee
        for img in employee_images:
            img = np.expand_dims(img, axis=0)  # Add batch dimension for prediction
            embedding = embedding_model.predict(img)  # Get the embedding for the image
            embeddings.append(embedding.squeeze())  # Remove the batch dimension

        # Average the embeddings of the employee (mean embedding)
        employee_embeddings[employee_name] = np.mean(embeddings, axis=0)

    # Save the embeddings to a file
    embeddings_file_path = 'model_grayscale/employee_embeddings.npy'
    np.save(embeddings_file_path, employee_embeddings)
    print(f"Saved employee embeddings to '{embeddings_file_path}'")

# Load dataset from the folder structure
images, labels, label_map = load_images_from_directory(DATASET_PATH)
images = images / 255.0  # Normalize images

# Now, call the function to generate and save the embeddings
generate_embeddings_for_all_employees(images, labels, label_map, embedding_model)


# 3.Perform face recognition

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from scipy.spatial.distance import cosine

@tf.keras.utils.register_keras_serializable()
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    positive_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    negative_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(positive_dist - negative_dist + alpha, 0.0)
    return tf.reduce_mean(loss)

employee_embeddings = np.load('model_grayscale/employee_embeddings.npy', allow_pickle=True).item()

embedding_model = tf.keras.models.load_model(
    'model_grayscale/embedding_model.keras',
    custom_objects={'triplet_loss': triplet_loss}
)

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (112, 112))
    image = image / 255.0
    return np.expand_dims(image, axis=0)

def get_embedding(image):
    return embedding_model.predict(image).squeeze()

def recognize_face(input_image_path, employee_embeddings, threshold=0.6):
    input_image = preprocess_image(input_image_path)
    input_embedding = get_embedding(input_image)

    min_distance = float('inf')
    recognized_name = "Unknown"
    for person_name, saved_embedding in employee_embeddings.items():
        distance = cosine(input_embedding, saved_embedding)
        if distance < min_distance:
            min_distance = distance
            recognized_name = person_name

    if min_distance < threshold:
        return recognized_name
    else:
        return "Unknown"

input_image_path = 'testing_data/testing_image_5.jpg'
recognized_name = recognize_face(input_image_path, employee_embeddings)
print(f"Recognized: {recognized_name}")
